In [ ]:
from ddm.data_handling import read_file
from ddm.processing import ddm_dask
from ddm.fitting import findMeanSqFFT_np, computeAB
from ddm.plotting import plotSingleExpFit, plotISF
from ddm.utils import print_available_ram, print_file_size

import numpy as np
import dask
import dask.array as da
from dask.diagnostics import ProgressBar

In [ ]:
print_available_ram()
print_file_size('../data/test_dataset_no_drift.tif')

In [ ]:
filename = '../data/test_dataset_no_drift.tif'
ddmData = read_file(filename, delayed=True, chunk_size=25)
ddmData

In [ ]:
from dask.distributed import Client

In [ ]:
client = Client(n_workers=4, threads_per_worker=1, memory_limit='2GB', processes=True)
client

In [ ]:
def calc_matrix_loop(data, tau):
    num_frames, height, width = data.shape
    img_fft_sq = da.zeros((width, height))  

    for jj in range(num_frames - tau):  
        img_diff = (data[jj + tau] - data[jj])  
        img_fft = da.fft.fft2(img_diff)
        img_fft_sq += (da.abs(img_fft) ** 2)

    fft_shift = da.fft.fftshift(img_fft_sq)
    return fft_shift


def calc_matrix_loop_opt(data, tau, num_frames):
    img_fft_sq = []

    for jj in range(num_frames - tau):  
        out = calc_ISF(data[jj], data[jj + tau])
        img_fft_sq.append(out)

    fft_shift = da.fft.fftshift(np.asarray(img_fft_sq))
    return fft_shift

In [ ]:
# def calc_ISF(img_diff):
#     """Calculate the Image Structure Function
#     """
#     return da.abs(da.fft.fft2(img_diff))**2

# out = []
# results = []
# for tau in range(1,10):
#     img_diff = ddmData.data[:-tau, :, :] - ddmData.data[tau:, :, :]
#     ISF = da.stack([calc_ISF(img) for img in img_diff])
#     out.append(da.fft.fftshift(da.sum(ISF, axis=0)))
#     results.append(dask.compute(*out))

In [ ]:
def calc_ddm(img_fft, tau):
    # img_fft = da.fft.fft2(data).astype("complex64")
    img_diff = img_fft[:-tau, :, :] - img_fft[tau:, :, :]
    img_fft_sq = da.abs(img_diff) ** 2
    img_sum = da.sum(img_fft_sq, axis=0)
    return da.fft.fftshift(img_sum)

In [ ]:
%%time

def calc_ddm(img_fft, tau):
    # img_fft = da.fft.fft2(data).astype("complex64")
    img_diff = img_fft[:-tau, :, :] - img_fft[tau:, :, :]
    img_fft_sq = da.abs(img_diff) ** 2
    img_sum = da.sum(img_fft_sq, axis=0)
    return da.fft.fftshift(img_sum)

results = []
img_fft = da.fft.fft2(ddmData.data).astype("complex64")
for tau in range(1,10):
    results.append(calc_ddm(img_fft, tau))

out = dask.compute(*results)

In [ ]:
%%time

def calc_ddm(data, tau):
    img_fft = da.fft.fft2(data).astype("complex64")
    img_diff = img_fft[:-tau, :, :] - img_fft[tau:, :, :]
    img_fft_sq = da.abs(img_diff) ** 2
    img_sum = da.sum(img_fft_sq, axis=0)
    return da.fft.fftshift(img_sum)

results = []
for tau in range(1,10):
    results.append(calc_ddm(ddmData.data, tau))

out = dask.compute(*results)

In [ ]:
# out = dask.compute(*results)

In [ ]:
np.asarray(out).shape

In [ ]:
client.close()